# Exposure bracketing using the sequencer feature of USB3 ace devices

In [4]:
import pypylon.pylon as py
import numpy as np

In [5]:
# open the first USB device
info = py.DeviceInfo()
info.SetDeviceClass("BaslerUsb")
cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice(info))

In [6]:
# this code only works for ace USB
if not cam.GetDeviceInfo().GetModelName().startswith("acA"):
    print("_This_ sequencer configuration only works to basler ace USB")

In [7]:
# open device
cam.Open()

## setup camera.
sequencer mode is not running if the auto-functions are active

In [8]:
cam.UserSetSelector = "Default"
cam.UserSetLoad.Execute()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\2017762402.py:1: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.UserSetSelector.Value = Default
  cam.UserSetSelector = "Default"


In [9]:
# mean exposure time 20ms
exp_0 = 2000
exp_1 = 20000
exp_2 = 200000

### activate chunks [ embedded data ]
This allows to read the exposuretime an image has been taken with from the image data

In [10]:
# enable camera chunk mode
cam.ChunkModeActive = True
# enable exposuretime chunk
cam.ChunkSelector = "ExposureTime"
cam.ChunkEnable = True

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\3045622392.py:2: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ChunkModeActive.Value = True
  cam.ChunkModeActive = True
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\3045622392.py:4: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ChunkSelector.Value = ExposureTime
  cam.ChunkSelector = "ExposureTime"
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\3045622392.py:5: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ChunkEnable.Value = True
  cam.ChunkEnable = True


### sequencer setup

In [11]:
cam.SequencerMode = "Off"
cam.SequencerConfigurationMode = "On"

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\2540021868.py:1: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerMode.Value = Off
  cam.SequencerMode = "Off"
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\2540021868.py:2: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerConfigurationMode.Value = On
  cam.SequencerConfigurationMode = "On"


### setup set 0

In [12]:
cam.SequencerSetSelector = 0

cam.ExposureTime = exp_0

cam.SequencerSetSave.Execute()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\1515968180.py:1: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerSetSelector.Value = 0
  cam.SequencerSetSelector = 0
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\1515968180.py:3: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 2000
  cam.ExposureTime = exp_0


In [10]:
### setup set 1

In [13]:
cam.SequencerSetSelector = 1

cam.ExposureTime = exp_1

cam.SequencerSetSave.Execute()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\1496113563.py:1: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerSetSelector.Value = 1
  cam.SequencerSetSelector = 1
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\1496113563.py:3: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 20000
  cam.ExposureTime = exp_1


In [14]:
### setup set 2

In [15]:
cam.SequencerSetSelector = 2

cam.ExposureTime = exp_2

# select that we jump to set 0 after this set
# path 1 is the _next_ path / path 0 would be _reset_ path
cam.SequencerPathSelector = 1
cam.SequencerSetNext = 0

cam.SequencerSetSave.Execute()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\332444998.py:1: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerSetSelector.Value = 2
  cam.SequencerSetSelector = 2
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\332444998.py:3: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 200000
  cam.ExposureTime = exp_2
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\332444998.py:7: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerPathSelector.Value = 1
  cam.SequencerPathSelector = 1
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\332444998.py:8: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerSetNext.Value = 0
  cam.SequencerSetNext = 0


### enable sequencer

In [16]:
cam.SequencerConfigurationMode = "Off"

# this will set the first sequencer set to set _0_ as side effect
cam.SequencerMode = "On"

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\183140513.py:1: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerConfigurationMode.Value = Off
  cam.SequencerConfigurationMode = "Off"
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7936\183140513.py:4: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.SequencerMode.Value = On
  cam.SequencerMode = "On"


### test capture with enabled sequencer mode

In [18]:
# grab 4 sets of 3 images
cam.StartGrabbingMax(20 * 3)

while cam.IsGrabbing():
    res = cam.RetrieveResult(1000)
    exp_time_chunk = res.ChunkDataNodeMap.ChunkExposureTime.Value
    print("%d\t%6.0f\t%6.2f" % (res.BlockID, exp_time_chunk, np.mean(res.Array)))
    # ... do something with the images

    # return to buffer queue
    res.Release()

cam.StopGrabbing()

0	  2000	  0.18
1	 20000	  4.84
2	200000	 48.60
3	  2000	  0.18
4	 20000	  4.85
5	200000	 48.60
6	  2000	  0.18
7	 20000	  4.85
8	200000	 48.60
9	  2000	  0.18
10	 20000	  4.85
11	200000	 48.60
12	  2000	  0.18
13	 20000	  4.85
14	200000	 48.60
15	  2000	  0.18
16	 20000	  4.85
17	200000	 48.60
18	  2000	  0.18
19	 20000	  4.85
20	200000	 48.60
21	  2000	  0.18
22	 20000	  4.85
23	200000	 48.60
24	  2000	  0.18
25	 20000	  4.85
26	200000	 48.60
27	  2000	  0.18
28	 20000	  4.85
29	200000	 48.60
30	  2000	  0.18
31	 20000	  4.85
32	200000	 48.60
33	  2000	  0.18
34	 20000	  4.85
35	200000	 48.60
36	  2000	  0.18
37	 20000	  4.85
38	200000	 48.60
39	  2000	  0.18
40	 20000	  4.85
41	200000	 48.60
42	  2000	  0.18
43	 20000	  4.85
44	200000	 48.61
45	  2000	  0.18
46	 20000	  4.85
47	200000	 48.61
48	  2000	  0.18
49	 20000	  4.85
50	200000	 48.61
51	  2000	  0.18
52	 20000	  4.85
53	200000	 48.60
54	  2000	  0.18
55	 20000	  4.85
56	200000	 48.61
57	  2000	  0.18
58	 20000	  4.85
59	2000

### Shutdown session after acquisition

This is required to rerun the notebook